In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sheet\\Desktop\\Briefly\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\sheet\\Desktop\\Briefly'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
import sys
import os

sys.path.append(os.path.abspath("src"))

from Briefly.constants import *  # Now it should work
from Briefly.utils.common import  read_yaml, create_directories
# this goes in configuration

In [7]:
class ConfigurationManager:           #This goes into configuration.py
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        """Initialize Configuration Manager and load YAML configs."""
        
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Ensure artifacts directory exists
        create_directories([self.config["artifacts_root"]])   #this part is same in ingestion and validation
        
        
     #only copy this part in configuration.py for validation   
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path= config.data_path,
            tokenizer_name= config.tokenizer_name,
        )
        return data_transformation_config
        
        
        

In [8]:
import os
from Briefly.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk


C:\Users\sheet\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-06-20 17:01:42,470: INFO: PyTorch version 2.5.1 available.]
[2025-06-20 17:01:42,482: INFO: TensorFlow version 2.19.0 available.]
[2025-06-20 17:01:42,489: INFO: JAX version 0.5.1 available.]


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
        
        
        
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(
        example_batch['dialogue'],
        max_length=1024,
        truncation=True,
        padding="max_length"  # Add padding
    )

        target_encodings = self.tokenizer(
        example_batch['summary'],
        max_length=128,
        truncation=True,
        padding="max_length"
    )
        return{
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
        
    def convert(self):
        dataset_samsum= load_from_disk(self.config.data_path)
        dataset_samsum_pt= dataset_samsum.map(self.convert_examples_to_features, batched = True)
        short_path = os.path.abspath("samsum_ds")  # or use a temp folder like "samsum_ds_tmp"
        dataset_samsum_pt.save_to_disk(short_path)
        import shutil
        shutil.move(short_path, os.path.join(self.config.root_dir, "samsum_dataset"))


In [10]:
#pipeline
!pip install protobuf
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation= DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2025-06-20 17:01:50,861: INFO: yaml file: config\config.yaml loaded successfully]
[2025-06-20 17:01:50,872: INFO: yaml file: params.yaml loaded successfully]
[2025-06-20 17:01:50,874: INFO: created directory at: artifacts]
[2025-06-20 17:01:50,878: INFO: created directory at: artifacts/data_transformation]



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 22293.46 examples/s]
